In [1]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
!pip install opencv-python scikit-learn matplotlib

Cloning into 'yolov5'...


In [2]:
cd C:\shriya\ims\ICU Monitoring System\yolov5

C:\shriya\ims\ICU Monitoring System\yolov5


In [5]:
!python train.py --img 640 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot change to 'C:\shriya\ims\ICU': No such file or directory
YOLOv5  2024-7-14 Python-3.9.12 torch-2.3.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, b

                   all         22         55    0.00476      0.137     0.0333    0.00806

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size

  0%|          | 0/4 [00:00<?, ?it/s]
       5/49         0G    0.09199    0.04525    0.04694         75        640:   0%|          | 0/4 [00:09<?, ?it/s]
       5/49         0G    0.09199    0.04525    0.04694         75        640:  25%|##5       | 1/4 [00:09<00:29,  9.89s/it]
       5/49         0G    0.08819    0.04988    0.04475         94        640:  25%|##5       | 1/4 [00:19<00:29,  9.89s/it]
       5/49         0G    0.08819    0.04988    0.04475         94        640:  50%|#####     | 2/4 [00:19<00:19,  9.93s/it]
       5/49         0G    0.08752    0.04932      0.045         81        640:  50%|#####     | 2/4 [00:29<00:19,  9.93s/it]
       5/49         0G    0.08752    0.04932      0.045         81        640:  75%|#######5  | 3/4 [00:29<00:09,  9.81s/it]
       5/49         0G    0.08345    0.04747    0.0

In [6]:
import torch
import cv2
import numpy as np
import time
from sklearn.cluster import KMeans

model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'C:\shriya\ims\ICU Monitoring System\yolov5\runs\train\exp\weights\best.pt')

def detect_persons(image):
    results = model(image)
    boxes = results.xyxy[0][:, :4].cpu().numpy()
    class_ids = results.xyxy[0][:, 5].cpu().numpy()
    return boxes, class_ids

def calculate_iou(box1, box2):
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])
    
    intersection_area = max(0, x2_min - x1_max + 1) * max(0, y2_min - y1_max + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou

def process_video(input_video_path, output_video_path, ground_truths=None):
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open input video.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev_frame)
    mask[..., 1] = 255  
    frame_count = 0
    patient_movements = []
    movement_detected = False

    total_prediction_time = 0
    correct_predictions = 0
    total_predictions = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 30 == 0:  
            print(f'Processing frame {frame_count}')

        start_time = time.time()
        person_boxes, class_ids = detect_persons(frame)
        end_time = time.time()

        prediction_time = end_time - start_time
        total_prediction_time += prediction_time

        categories = ['nurse','family member','intensivist','patient']

        for box, class_id in zip(person_boxes, class_ids):
            x1, y1, x2, y2 = map(int, box)
            category = categories[int(class_id)]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, category, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
            if category == 'patient':
                patient_movements.append((x1, y1, x2, y2))
            
            if ground_truths:
                # Calculate accuracy
                for gt_box, gt_label in ground_truths.get(frame_count, []):
                    if category == gt_label:
                        iou = calculate_iou(box, gt_box)
                        if iou > 0.5:
                            correct_predictions += 1
                            break
                total_predictions += 1

        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        mask[..., 0] = angle * 180 / np.pi / 2
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        rgb_flow = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

        for x1, y1, x2, y2 in patient_movements:
            movement = np.mean(magnitude[y1:y2, x1:x2])
            if movement > 1.5:  
                movement_detected = True

        if movement_detected:
            cv2.putText(frame, 'Movement Detected', (50, height - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            movement_detected = False  

        prev_gray = gray

        out.write(frame)

    cap.release()
    out.release()
    
    avg_prediction_time = total_prediction_time / frame_count if frame_count else 0
    accuracy = correct_predictions / total_predictions if total_predictions else 0
    
    print(f'Processed video saved to {output_video_path}')
    print(f'Average prediction time per frame: {avg_prediction_time:.4f} seconds')
    if ground_truths:
        print(f'Accuracy: {accuracy:.4f}')
    else:
         print('Accuracy calculation skipped due to lack of ground truth data.')


ground_truths = {
#     Format: frame_number: [(bbox, label)]
#     Example: 1: [((x1, y1, x2, y2), 'patient')]
 }
input_video_path = r'C:\shriya\ims\ICU Monitoring System\Sample\icu-scene.mp4'
output_video_path = r'C:\shriya\ims\ICU Monitoring System\Sample\output_video.mp4'
process_video(input_video_path, output_video_path, ground_truths)


Using cache found in C:\Users\saish/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-6-24 Python-3.9.12 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Processing frame 30
Processing frame 60
Processing frame 90
Processing frame 120
Processing frame 150
Processing frame 180
Processing frame 210
Processing frame 240
Processing frame 270
Processing frame 300
Processing frame 330
Processing frame 360
Processing frame 390
Processing frame 420
Processing frame 450
Processing frame 480
Processing frame 510
Processing frame 540
Processing frame 570
Processing frame 600
Processing frame 630
Processing frame 660
Processing frame 690
Processing frame 720
Processing frame 750
Processing frame 780
Processing frame 810
Processing frame 840
Processing frame 870
Processing frame 900
Processed video saved to C:\shriya\ims\ICU Monitoring System\Sample\output_video.mp4
Average prediction time per frame: 0.1101 seconds
Accuracy calculation skipped due to lack of ground truth data.
